# SQL II: Grouping and Joining

_September 21, 2020_

Agenda:
- Practice with Grouping in SQLite
- Practice with different types of Joins in SQLite

<img src = "https://media.giphy.com/media/3oKIPnAiaMCws8nOsE/giphy.gif">

In [5]:
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np

## Part I. Grouping
Just like pandas, Sql support different types of grouping statements for performing aggregate functions and allow us to calculate various statistics for data.

Syntax:
`SELECT (*) FROM tables GROUP BY column`

In [6]:
# instantiate some data
auto = pd.read_csv('auto-mpg.csv')
tips = sns.load_dataset('tips')
small_quiz = pd.DataFrame({"students":["Raizel","Mitch","Eric"],
                          "projects":[1,2,1],
                          "grades":np.random.randint(80,100,3)})
small_grades = pd.DataFrame({"students":["Mike W","Mitch","Mike R"],
                            "quiz_score":np.random.randint(0,10,3)})

In [7]:
conn = sqlite3.connect('aggregate.db')
c = conn.cursor()

In [8]:
# insert these datasets into our db
auto.to_sql('auto', con=conn)
tips.to_sql('tips', con=conn)
small_grades.to_sql('small_grade', con=conn)
small_quiz.to_sql('small_quiz',con=conn)

/Users/brendanferris/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


ValueError: Table 'auto' already exists.

In [11]:
auto

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [12]:
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [13]:
small_quiz

,students,projects,grades
0,Raizel,1,83
1,Mitch,2,97
2,Eric,1,90


In [15]:
small_grades

,students,quiz_score
0,Mike W,0
1,Mitch,5
2,Mike R,2


In [31]:
# for the tips df/table, get the total number of man and woman

pd_query_1 = tips.groupby(['sex']).size()

# tips.sex.value_counts() #<<<<---can also get same info without groupby---#

In [32]:
pd_query_1

sex
Male      157
Female     87
dtype: int64

Syntax: `SELECT ___ FROM ___ GROUP BY ___` 

In [33]:
# in sql 
query = """SELECT sex, COUNT(sex) FROM tips GROUP BY sex"""
c.execute(query).fetchall()

[('Female', 87), ('Male', 157)]

In [40]:
# using pandas, select the average amount tipped for time
pd_query_2 = tips.groupby(['time'])['tip'].mean()
pd_query_2

time
Lunch     2.728088
Dinner    3.102670
Name: tip, dtype: float64

In [41]:
# use sql to recreate this query
query = """SELECT time, AVG(tip) FROM tips GROUP BY time"""
c.execute(query).fetchall()

[('Dinner', 3.102670454545454), ('Lunch', 2.7280882352941176)]

We can also use `GROUP BY` with certain conditions, in conjunction with filtering and ordering.

You can use :

`SELECT ___ FROM ___ WHERE ___ GROUP BY ___ `

`SELECT ___ FROM ___ GROUP BY ___ HAVING ____ `

In [52]:
#^ notice the syntax: placement of WHERE and HAVING depending on context.

In [50]:
# use pandas, get the average total bills for a party larger than 2 for every size of party
condition = tips[tips['size'] > 2]
pd_query_3 = condition.groupby(['size'])['total_bill'].mean()
pd_query_3

# using pandas

size
3    23.277632
4    28.613514
5    30.068000
6    34.830000
Name: total_bill, dtype: float64

In [ ]:
# use sql to recreate this query
query = None
c.execute(query).fetchall()

In [77]:
# your turn - use pandas, get the maximum value of total bill for female non smoker

tips[(tips.smoker == 'No') & (tips.sex == 'Female')].total_bill.max()

# using group by?
v = tips[tips.smoker == 'No'].groupby(['sex']).max()
# v.loc['Female'][0]

35.83

In [81]:
tips.groupby(['sex', 'smoker']).total_bill.max()['Female']['No'] # using a groupby multi-index series

35.83

In [ ]:
# how to get this without using group by?


In [84]:
# use sql to recreate this query
query = """SELECT total_bill FROM tips WHERE smoker = 'No' AND sex = 'Female' ORDER BY total_bill DESC Limit 1"""
c.execute(query).fetchall()

[(35.83,)]

In [ ]:
# your turn - use pandas, get the average tip values for different size groups, and show top 2
# most amount of average tips


In [ ]:
# use sql 
query = None
c.execute(query).fetchall()

## Part II. Join

Based on this diagram, how would you explain the different types of joins?

<img src = 'sql-joins.png' width = 400>

In [85]:
print(small_grades)
print(small_quiz)

  students  quiz_score
0   Mike W           0
1    Mitch           5
2   Mike R           2
  students  projects  grades
0   Raizel         1      83
1    Mitch         2      97
2     Eric         1      90


In [86]:
# left join - join small quiz with small grades, on student name
small_grades.merge(small_quiz, left_on = 'students', right_on = 'students', how = "left")

# if i run this command, what would happen? how many observations/record?

,students,quiz_score,projects,grades
0,Mike W,0,NaN,NaN
1,Mitch,5,2.0,97.0
2,Mike R,2,NaN,NaN


In [92]:
# sql 
query = """SELECT * FROM small_grade LEFT JOIN small_quiz ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

[(0, 'Mike W', 1, None, None, None, None),
 (1, 'Mitch', 1, 1, 'Mitch', 2, 99),
 (2, 'Mike R', 8, None, None, None, None)]

In [ ]:
# inner join and get the student's grades
small_grades.merge(small_quiz, on = 'students')

In [ ]:
query = """SELECT * FROM small_quiz JOIN small_grade ON small_grade.students = small_quiz.students"""
c.execute(query).fetchall()

In [ ]:
# your turn - use right join and get all entries

# pandas

In [ ]:
# sql 

### Level up 
1. Subqueries (including anti-joins)
2. Get median (using OFFSET)

In [ ]:
# anti-join :get all the entries that are in the left table but not in the right table 
query = """SELECT left_table.id FROM `left table` WHERE left_table.id not in (SELECT id FROM right_table) """

In [ ]:
# get median - use offset 
query = """SELECT col FROM table ORDER BY col LIMIT 1 OFFSET (SELECT COUNT(*) FROM MyTable) / 2"""